In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import tensorflow_datasets as tfds

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
#Load the tensorflow dataset, subset 1 out of 10, version 2.5.0, training split
ds_builder = tfds.builder("clic_edm_ttbar_pf/1:2.5.0", data_dir="/home/joosep/")
ds = ds_builder.as_data_source("train")
ds.dataset_info

In [ ]:
iev = 0

Xelems = []
ytargets = []
ycands = []

for elem in ds:
    #print the first few events
    if iev<2:
        print("event", iev, elem.keys())
        for k in elem.keys():
            print(k, elem[k])
    Xelems.append(elem["X"])
    ytargets.append(elem["ytarget"])
    ycands.append(elem["ycand"])
    iev += 1
    if iev>1000:
        break

#concatenate across event dimension
Xelems_flat = np.concatenate(Xelems, axis=0)
ytargets_flat = np.concatenate(ytargets, axis=0)
ycands_flat = np.concatenate(ycands, axis=0)

In [ ]:
#pick tracks or clusters
mask_tracks = Xelems_flat[:, 0]==1
mask_clusters = Xelems_flat[:, 0]==2

#keep only the rows where both the PF particle and the target particle exist
mask_target_particle = ytargets_flat[:, 0]!=0
mask_pf_particle = ycands_flat[:, 0]!=0

In [ ]:
plt.hist(
    (ytargets_flat[:, 2]/Xelems_flat[:, 1])[mask_target_particle & mask_tracks],
    bins=np.linspace(0,2,500)
);
plt.yscale("log")
plt.xlabel("target particle / track $p_T$ ratio")
plt.ylabel("Number of particles / bin")

In [ ]:
plt.hist(
    (ytargets_flat[:, 2]/Xelems_flat[:, 1])[mask_target_particle & mask_clusters],
    bins=np.linspace(0,2,500)
);
plt.yscale("log")
plt.xlabel("target particle / cluster $p_T$ ratio")
plt.ylabel("Number of particles / bin")

In [ ]:
plt.hist(
    (ycands_flat[:, 2]/ytargets_flat[:, 2])[mask_target_particle & mask_pf_particle],
    bins=np.linspace(0,2,500)
);
plt.yscale("log")
plt.xlabel("PF particle / target particle $p_T$ ratio")
plt.ylabel("Number of particles / bin")